In [1]:
import numpy as np
import jax
import jax.numpy as jnp
import healpy as hp
from functools import partial

In [2]:
jax.config.update("jax_enable_x64", True)

In [3]:
nside = 64
lmax = 2*nside

almg = hp.synalm(np.ones(lmax + 1), lmax)
almc = hp.synalm(np.ones(lmax + 1), lmax)

In [4]:
# mmax = lmax

# def JAX_alm_getidx(lmax, ell, m_idx):
#     return m_idx*(2*lmax+1-m_idx)//2+ell

# @partial(jax.jit, static_argnums=(2))
# def JAX_almxfl(alm, c_ell_x_, lmax, mmax=None):

#     if mmax is None:
#         mmax = lmax
    
#     @partial(jax.jit, static_argnums=(1))
#     def map_multiplication_per_m(carry, m_idx):
#         if m_idx.astype(int) == -1:
#             return carry, m_idx
#         alm_carry, c_ell_x_alm, ell  = carry
#         idx_lm = JAX_alm_getidx(lmax, ell, m_idx)
#         return (alm_carry[idx_lm]*c_ell_x_alm, c_ell_x_alm, ell), alm_carry[idx_lm]*c_ell_x_alm

#     @partial(jax.jit, static_argnums=(1))
#     def map_multiplication_per_ell(carry, ell):
#         alm_carry = carry
#         c_ell_x_alm = jnp.where(ell < lmax+1, c_ell_x_[ell], 0.)
        
#         # max_m_ell = jnp.where(ell < mmax+1, ell, mmax)
#         max_m_ell = jnp.minimum(ell, mmax)
#         print("Test", ell, max_m_ell, flush=True)
#         arange_dynamic = jnp.where(jnp.arange(lmax + 1)<max_m_ell+1, jnp.arange(mmax + 1), -1)
#         carry_output, m_idx = jax.lax.scan(map_multiplication_per_m, (alm_carry, c_ell_x_alm, ell), arange_dynamic)
#         alm_carry, c_ell_x_alm, ell = carry_output
#         return alm_carry, ell

#     alm_in = jnp.copy(alm)
#     alms_output, _ = jax.lax.scan(map_multiplication_per_ell, alm_in, jnp.arange(lmax + 1))
#     return alm

In [94]:
mmax = lmax

def JAX_alm_getidx(lmax, ell, m_idx):
    return m_idx*(2*lmax+1-m_idx)//2+ell

@partial(jax.jit, static_argnames=("lmax"))
def JAX_almxfl(alm, c_ell_x_, lmax):

    # if mmax is None:
    #     mmax = lmax
    
    # all_m_idx = jax.vmap(lambda m_idx:m_idx*(2*lmax+1-m_idx)//2)(jnp.arange(mmax+1))
    all_m_idx = jax.vmap(lambda m_idx:m_idx*(2*lmax+1-m_idx)//2)(jnp.arange(lmax+1))

    alm_carry = jnp.copy(alm)
    def func_scan(carry, ell):
        _alm_carry = carry
        # mask_m = jnp.where(jnp.arange(mmax+1)<=ell, 1, 0)
        # inverse_mask_m = jnp.where(jnp.arange(mmax+1)<=ell, 0, 1)
        mask_m = jnp.where(jnp.arange(lmax+1)<=ell, c_ell_x_[ell], 1)
        # inverse_mask_m = jnp.where(jnp.arange(lmax+1)<=ell, 0, 1)
        # _alm_carry = _alm_carry.at[all_m_idx+ell].set(_alm_carry[all_m_idx+ell]*c_ell_x_[ell]*mask_m + _alm_carry[all_m_idx+ell]*inverse_mask_m)
        _alm_carry = _alm_carry.at[all_m_idx+ell].set(_alm_carry[all_m_idx+ell]*mask_m)
        return _alm_carry, ell

    # alms_output = jax.vmap(fmap)(jnp.arange(lmax+1))
    alms_output, _ = jax.lax.scan(func_scan, alm_carry, jnp.arange(lmax+1))
    return alms_output
    

In [116]:
8385/(lmax+1)

65.0

In [95]:
all_m_idx = jax.vmap(lambda m_idx:m_idx*(2*lmax+1-m_idx)//2)(jnp.arange((lmax+1)//2))

In [96]:
all_m_idx.shape

(64,)

In [97]:
all_m_idx

Array([   0,  128,  255,  381,  506,  630,  753,  875,  996, 1116, 1235,
       1353, 1470, 1586, 1701, 1815, 1928, 2040, 2151, 2261, 2370, 2478,
       2585, 2691, 2796, 2900, 3003, 3105, 3206, 3306, 3405, 3503, 3600,
       3696, 3791, 3885, 3978, 4070, 4161, 4251, 4340, 4428, 4515, 4601,
       4686, 4770, 4853, 4935, 5016, 5096, 5175, 5253, 5330, 5406, 5481,
       5555, 5628, 5700, 5771, 5841, 5910, 5978, 6045, 6111], dtype=int64)

In [98]:
m, dt_der1, dp_der1 = hp.alm2map_der1(almg, nside)

In [99]:
%%time
for i in range(1000):
    alm_spin = hp.almxfl(
        almg,
        np.array(
            [np.sqrt(l * (l + 1.0)) for l in range(0, lmax + 1)]
        ),
        inplace=False,
    )


CPU times: user 349 ms, sys: 101 ms, total: 450 ms
Wall time: 172 ms


In [100]:
%%time
for i in range(1000):
    alm_spin_JAX = JAX_almxfl(
        almg,
        np.array(
            [np.sqrt(l * (l + 1.0)) for l in range(0, lmax + 1)]
        ),
        lmax,
    )


TypeError: mul got incompatible shapes for broadcasting: (8385,), (129,).

In [101]:
jitted_almxfl = jax.jit(JAX_almxfl, static_argnames=("lmax",))

In [102]:
%%time
alm_spin_JAX = jitted_almxfl(
    np.array(almg),
    np.array(
        [np.sqrt(l * (l + 1.0)) for l in range(0, lmax + 1)]
    ),
    lmax,
)


TypeError: mul got incompatible shapes for broadcasting: (8385,), (129,).

In [103]:
%%time
for i in range(1000):
    alm_spin_JAX = jitted_almxfl(
        np.array(almg),
        np.array(
            [np.sqrt(l * (l + 1.0)) for l in range(0, lmax + 1)]
        ),
        lmax,
    )


TypeError: mul got incompatible shapes for broadcasting: (8385,), (129,).

In [104]:
dt_spin, dp_spin = hp.alm2map_spin(
    [alm_spin, alm_spin * 0.0], nside, 1, lmax
)

dt_spin, dp_spin = hp.alm2map_spin(
    [alm_spin_JAX, alm_spin_JAX * 0.0], nside, 1, lmax
)

np.testing.assert_array_almost_equal(dt_der1, dt_spin, decimal=8)
np.testing.assert_array_almost_equal(dp_der1, dp_spin, decimal=8)

In [105]:
alm_spin_JAX-alm_spin

Array([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j], dtype=complex128)

In [106]:
alm_spin_JAX

Array([   0.          +0.j        ,   -0.31766397  +0.j        ,
          2.8292244   +0.j        , ...,  -19.90241108 -93.94287318j,
         24.41376751 +75.08435299j, -130.97559829+114.87294981j],      dtype=complex128)

In [107]:
alm_spin

array([   0.          +0.j        ,   -0.31766397  +0.j        ,
          2.8292244   +0.j        , ...,  -19.90241108 -93.94287318j,
         24.41376751 +75.08435299j, -130.97559829+114.87294981j])